In [2]:
import csv
import statistics
import time

# Load data from CSV
data = []
with open("DARWIN.csv", "r") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        data.append([float(val) for val in row])

totalObjCount = len(data)
print('Total no. of objects=', totalObjCount)
totalLen = len(data[0])
print('Total no. of attributes=', totalLen)

# Define functions
def attrBroadcast(i):
    attr = [col[i] for col in data]
    return attr

def sNorm(a, b):
    return a + b

def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier

def findSim(x, y, std):
    valA = x - y + std
    valA = valA / std

    valB = y - x + std
    valB = valB / std

    s = min(valA, valB)
    sRes = max(s, 0)
    sRes = truncate(sRes, 2)
    return sRes

def similarityMatrix_ID(descId, snorm, R):
    listA = []
    decValue = decisionVariable
    attributeValue = attrBroadCastVariable
    for attrId in range(0, len(attributeValue)):
        if decValue[descId] != decValue[attrId] and descId < attrId:
            listA.append((descId, attrId, snorm, R))
    return listA

def similarityMatrix(x1, x2, Reduct, std):
    attrVal = attrBC_Var
    dSimVal = 1 - findSim(attrVal[x1], attrVal[x2], std)
    norm = sNorm(dSimVal, Reduct)
    norm = truncate(norm, 2)
    return (x1, x2, norm, Reduct)

def similarityBackward(x1, x2, Reduct, std):
    attrVal = attrBC_Var2
    dSimVal = 1 - findSim(attrVal[x1], attrVal[x2], std)
    norm = Reduct - dSimVal
    norm = truncate(norm, 2)
    return (x1, x2, norm, Reduct)

# Main
st = time.time()

decisionVariable = [row[0] for row in data]
reductOutput = []
d = {}

posReg, currPosReg = 0.0, 0.0
snorm, R = 0, 0

arrBroadCast = attrBroadcast(1)
attrBroadCastVariable = arrBroadCast

resultFirst = []
for descId in range(totalObjCount):
    resultFirst.extend(similarityMatrix_ID(descId, snorm, R))

for i in range(1, totalLen - 1):
    BC_List = attrBroadcast(i)
    attrBC_Var = BC_List
    std_dev = statistics.stdev(BC_List)
    if std_dev == 0:
        continue
    d[i] = std_dev

    resultRdd = []
    for x in resultFirst:
        resultRdd.append(similarityMatrix(x[0], x[1], x[3], std_dev))

    combinedRdd = []
    for x in resultRdd:
        combinedRdd.extend([(x[0], x[2]), (x[1], x[2])])

    storeKeyVal = {}
    for col in combinedRdd:
        key = col[0]
        val = min(1, col[1])
        if key not in storeKeyVal:
            storeKeyVal[key] = val
        else:
            storeKeyVal[key] = min(storeKeyVal[key], val)

    values1 = list(storeKeyVal.values())
    currPosReg1 = sum(values1)

    if posReg < currPosReg1:
        reductOutput.append(i)
        posReg = currPosReg1
        finalRes = []
        for x in resultRdd:
            finalRes.append((x[0], x[1], 0.0, x[2]))
        resultFirst = finalRes

    if posReg == totalObjCount:
        break

print("reductOutput after forward process :: ", reductOutput)

for i in list(reductOutput):
    BC_List = attrBroadcast(i)
    attrBC_Var2 = BC_List
    std_dev2 = d.get(i)
    if std_dev2 == 0:
        continue

    resultRdd = []
    for x in resultFirst:
        resultRdd.append(similarityBackward(x[0], x[1], x[3], std_dev2))

    combinedRdd = []
    for x in resultRdd:
        combinedRdd.extend([(x[0], x[2]), (x[1], x[2])])

    storeKeyVal = {}
    for col in combinedRdd:
        key = col[0]
        val = min(1, col[1])
        if key not in storeKeyVal:
            storeKeyVal[key] = val
        else:
            storeKeyVal[key] = min(storeKeyVal[key], val)

    values1 = list(storeKeyVal.values())
    currPosReg1 = sum(values1)

    if posReg == currPosReg1:
        reductOutput.remove(i)
        finalRes = []
        for x in resultRdd:
            finalRes.append((x[0], x[1], 0.0, x[2]))
        resultFirst = finalRes

print("reductOutput after backward elimination :: ", reductOutput)
et = time.time()

elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')





Total no. of objects= 174
Total no. of attributes= 450
reductOutput after forward process ::  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
reductOutput after backward elimination ::  [1, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13]
Execution time: 1.9146697521209717 seconds
